Scraping Amazon for headphones data

In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import time
import csv
import random

import keepa 


In [7]:
amazon_url = "https://www.amazon.com/Logitech-Headphones-Canceling-Microphone-Chromebook/dp/B0F5HC4FRL/ref=sr_1_2_sspa?crid=2RXU5QF0WCPPX&dib=eyJ2IjoiMSJ9.UJqi8EFNPupulh7gnfbeTbQvG-f90mOPqqdBqEKI18JmF0n3qo1z-HaXJl6BKUn0SKDIafKt5Tv7Aj2QMVBz1RztGNkYXR9915VPI5XundQEV_-mrXhK-Nx75dgZsKt0Pzx-FT2s6S32E1DxJNr9BZ1UtBludTcqJ7ybsnK3vJPRuFPGVCt_kwl-I4qzOhB8B6hci6ALh47ePaRxvLYhXrj3C_s1wGvbA9SR9Pr-e6I.VeX40Vrk_DyZ7m9EhvjqAaWUXx5jDT5RKlDwi0oKXPw&dib_tag=se&keywords=headphones&qid=1756569456&sprefix=headphon%2Caps%2C320&sr=8-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1"
base_url = "https://www.amazon.com/"

In [8]:
HEADERS = {
    "Accept" : "*/*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36"
}

In [ ]:
search = "headphones"
max_products = 900

product_links = []

for page in range(1, 100):  
    if len(product_links) >= max_products:
        break

    url = f"{base_url}/s?k={search}&page={page}"
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    results = soup.find_all("a", class_="a-link-normal s-no-outline")
    for a in results:
        link = a.get("href")
        if link and "/dp/" in link:
            full_link = base_url + link.split("?")[0]
            if full_link not in product_links:
                product_links.append(full_link)

    print(f"Page {page}: collected {len(product_links)} links")
    time.sleep(random.uniform(2, 5))  # avoid detection



Page 1: collected 17 links
Page 2: collected 34 links
Page 3: collected 51 links
Page 4: collected 68 links
Page 5: collected 85 links
Page 6: collected 102 links
Page 7: collected 119 links
Page 8: collected 136 links
Page 9: collected 153 links
Page 10: collected 170 links
Page 11: collected 187 links
Page 12: collected 203 links
Page 13: collected 220 links
Page 14: collected 237 links
Page 15: collected 254 links
Page 16: collected 271 links
Page 17: collected 288 links
Page 18: collected 304 links
Page 19: collected 321 links
Page 20: collected 324 links
Page 21: collected 325 links
Page 22: collected 326 links
Page 23: collected 327 links
Page 24: collected 327 links
Page 25: collected 328 links
Page 26: collected 329 links
Page 27: collected 330 links
Page 28: collected 331 links
Page 29: collected 332 links
Page 30: collected 333 links
Page 31: collected 334 links
Page 32: collected 334 links
Page 33: collected 335 links
Page 34: collected 336 links
Page 35: collected 337 links

In [13]:
data = []
for i, url in enumerate(product_links[:max_products]):
    try:
        res = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        # Product name
        title = soup.find("span", id="productTitle")
        title = title.get_text(strip=True) if title else "N/A"

        # Price
        price_span = soup.find("span", class_="a-price-whole")
        price = price_span.get_text(strip=True) if price_span else "N/A"

        data.append([title, price, url])
        print(f"[{i+1}] {title[:50]}... | Price: {price}")
        time.sleep(random.uniform(2, 6))  # delay
    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")



  0%|          | 0/1 [16:06<?, ?it/s]


[1] Sony WH-CH520 Wireless Headphones Bluetooth On-Ear... | Price: 7.
[2] BERIBES Bluetooth Headphones Over Ear, 65H Playtim... | Price: 18.
[3] Case Compatible with JBL Tune 510BT 520BT 720BT 77... | Price: 9.
[4] N/A... | Price: N/A
[5] Sony WH-1000XM4 Wireless Premium Noise Canceling O... | Price: 198.
[6] Bose QuietComfort Bluetooth Headphones, Wireless H... | Price: 229.
[7] Soundcore Anker Life Q20 Hybrid Active Noise Cance... | Price: 18.
[8] UGEE Inspiration Open Ear Headphones FS01, Bluetoo... | Price: 39.
[9] Wireless Earbuds, Sports Bluetooth Headphones, 80H... | Price: 22.
[10] Wireless Earbuds, ENC Noise Cancelling Sport Ear B... | Price: 25.
[11] Beats Studio Pro - Wireless Bluetooth Noise Cancel... | Price: 199.
[12] Sony WH-CH720N Noise Canceling Wireless Headphones... | Price: 8.
[13] NDO Wireless Earbuds Bluetooth 5.3 Headphones Spor... | Price: 22.
[14] JBL Tune 510BT - Bluetooth headphones with up to 4... | Price: 39.
[15] ZIUTY Wireless Earbuds Bluetooth Headphones

In [14]:
# Step 3: Save to CSV
with open("amazon_products.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Product Name", "Price", "URL"])
    writer.writerows(data)

print("📂 Saved results to amazon_products.csv")

📂 Saved results to amazon_products.csv


In [15]:
df = pd.read_csv("amazon_products.csv")
df.head()

,Product Name,Price,URL
0,Sony WH-CH520 Wireless Headphones Bluetooth On...,7.,https://www.amazon.com//Sony-WH-CH520-Headphon...
1,"BERIBES Bluetooth Headphones Over Ear, 65H Pla...",18.,https://www.amazon.com//BERIBES-Bluetooth-Head...
2,Case Compatible with JBL Tune 510BT 520BT 720B...,9.,https://www.amazon.com//Compatible-WH-CH720N-C...
3,NaN,NaN,https://www.amazon.com/https://aax-us-east-ret...
4,Sony WH-1000XM4 Wireless Premium Noise Canceli...,198.,https://www.amazon.com//Sony-WH-1000XM4-Cancel...


In [ ]:
# response = requests.get(amazon_url, headers=HEADERS)


# url = "https://www.amazon.com/dp/B0F5HC4FRL"
# options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # run in background
# options.add_argument("user-agent=Mozilla/5.0 ... Safari/537.36")

# driver = webdriver.Chrome(options=options)
# driver.get(url)
# time.sleep(2)  # wait for JS to load

# soup = BeautifulSoup(driver.page_source, "html.parser")

# prices = []
# for tag in soup.find_all("span", class_="a-price"):
#     whole = tag.find("span", class_="a-price-whole")
#     fraction = tag.find("span", class_="a-price-fraction")
#     if whole and fraction:
#         prices.append(whole.get_text(strip=True) + fraction.get_text(strip=True))

# print(prices)
# driver.quit()



['65.99', '29.99', '69.97', '55.99', '13.28', '15.99', '24.99', '20.49', '27.99', '29.41', '19.99', '29.99', '69.99', '32.99', '295.00', '59.99', '59.99', '77.96', '27.98', '29.99', '21.99', '34.99', '49.99', '29.99', '29.41', '19.99', '20.49']
